In [75]:
from classiq import *
import numpy as np
import scipy as sc

In [76]:
#constants 
k = 7
t = 0.767
A = np.array([[0, 1], [-1, 0]])
x_0 = [1, 1]
x_0_norm = np.linalg.norm(x_0)
T = int(np.log2(k + 1))
# A_norm = np.linalg.norm(A)
vs1 = []
c = 0

In [77]:
def VS1():
    global vs1
    global c
    c_m = np.zeros(k + 1)
    m_factorial = 1
    for i in range(k + 1):
        c_m[i] = (x_0_norm * (pow(t, i))) / m_factorial
        c += c_m[i]
        m_factorial *= (i + 1)

    c = np.sqrt(c)


    # Householder matrix
    e = np.zeros(k + 1)
    e[0] = 1
    w = np.subtract([np.sqrt(c_m[i]) / c for i in range(k + 1)], e)        
    vs1 = np.subtract(np.identity(k + 1), np.multiply(2 * (1 / np.inner(w, w)), np.outer(w, w)))


In [78]:
def evolution_operator():
    u_m = [np.array([[1, 0], [0, 1]])]
    for i in range(1, k + 1):
        u_m.append(u_m[i - 1] @ A)
    U = sc.linalg.block_diag(*u_m)
    return U

In [79]:
u_m = [np.array([[1, 0], [0, 1]])]
for i in range(1, k + 1):
    u_m.append(u_m[i - 1] @ A ) 
U = sc.linalg.block_diag(*u_m)

In [80]:
VS1()

In [81]:
@qfunc
def encoding(n_arr:QArray[QBit]):
    prob = [ 1 / 2, 1 / 2]
    inplace_prepare_state(prob, 0.01, n_arr[0])
    
    # H(n_arr[0])

    #VS1()
    unitary(vs1, n_arr[1:T + 1])

In [82]:
def bit_reverse_index(index, num_qubits):
    """Compute the bit-reversed index for a given number of qubits."""
    binary = f"{index:0{num_qubits}b}"  # Convert to binary with leading zeros
    reversed_binary = binary[::-1]      # Reverse the string
    return int(reversed_binary, 2)      # Convert back to integer

def reverse_unitary_matrix(U):
    """Rearrange a unitary matrix for reversed qubit order."""
    num_qubits = int(np.log2(U.shape[0]))  # Determine the number of qubits
    size = 2 ** num_qubits                 # Matrix size (2^n x 2^n)
    
    # Create a new matrix with permuted indices
    U_reversed = np.zeros_like(U)
    for i in range(size):
        for j in range(size):
            U_reversed[bit_reverse_index(i, num_qubits), bit_reverse_index(j, num_qubits)] = U[i, j]
    
    return U_reversed

In [83]:
@qfunc
def evolution(n_arr:QArray[QBit]):
   
     U = evolution_operator()
    # # U = reverse_unitary_matrix(U)
     unitary(U, n_arr)

In [84]:
@qfunc
def decoding(n_arr:QArray[QBit]):
    ws1 = vs1.T
    unitary(ws1, n_arr[1:T + 1])

In [88]:
@qfunc
def main(n_arr : Output[QArray[QBit, T+1]],x : Output[QBit]):
    allocate(n_arr)
    allocate(x)
    encoding(n_arr)
    evolution(n_arr)
    decoding(n_arr)

In [90]:
(c * c) * 0.4643252445

1.4139710156378262

In [89]:
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/2ufdCbtU4iDviFdYkDyalzZ1C1k?login=True&version=0.72.1
